# Rutina de análisis inicial y de resultados de datos etiquetados durante QC a EMA

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

El análisis de resultados se incluye dentro del documento de diagnóstico de series temporales.
___
**Librerías:**

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.image as mpimg
import glob
import os
import statistics
from matplotlib.ticker import FuncFormatter
from scipy import stats

_____

### Longitud y continuidad series de datos de EMA - Gráficas

In [19]:
# Path to the directory containing the CSV files
data_directory = "../OE_3_QC_Variables/3_TemperaturaSuelo/TS50/RawUnmodified_TS50/"
# List to hold dataframes
dataframes = []

# Load each CSV file into a dataframe and add it to the list
for filename in os.listdir(data_directory):
    if filename.endswith(".csv"):
        try:
            df = pd.read_csv(os.path.join(data_directory, filename), parse_dates=['Fecha'], encoding='latin-1')

            # Se verifica si 'Estado' existe y se aplica filtro
            if 'Estado' in df.columns:
                df = df[~df['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO0','0PAT','0PER']]))]

            # Only proceed if the DataFrame is not empty after filtering
            if not df.empty:
                df = df.copy()
                df.set_index('Fecha', inplace=True)
                df['Presence'] = 1  # Add a column to indicate data presence
                dataframes.append(df)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

# Se muestran los headers y fechas por archivo para ver estructura - Para caso de ejemplo
dataframes_info = [(df.head(), df.index.min(), df.index.max()) for df in dataframes]
dataframes_info

C:\Users\palvarez\AppData\Local\Temp\ipykernel_23112\3645912064.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(os.path.join(data_directory, filename), parse_dates=['Fecha'], encoding='latin-1')


[(                     Unnamed: 0   Station               Name  Sensor  Valor  \
  Fecha                                                                         
  2005-08-31 12:00:00       41933  11105020  CARMEN DEL DARIEN     243   24.0   
  2005-08-31 13:00:00       41934  11105020  CARMEN DEL DARIEN     243   18.6   
  2005-08-31 14:00:00       41935  11105020  CARMEN DEL DARIEN     243   21.4   
  2005-08-31 15:00:00       41936  11105020  CARMEN DEL DARIEN     243   19.6   
  2005-09-26 14:00:00       41937  11105020  CARMEN DEL DARIEN     243   37.1   
  
                       Presence  
  Fecha                          
  2005-08-31 12:00:00         1  
  2005-08-31 13:00:00         1  
  2005-08-31 14:00:00         1  
  2005-08-31 15:00:00         1  
  2005-09-26 14:00:00         1  ,
  Timestamp('2005-08-31 12:00:00'),
  Timestamp('2019-07-29 06:27:00')),
 (                     Unnamed: 0   Station    Name  Sensor  Valor  Presence
  Fecha                                  

In [29]:
# Esta si
# Ordenar lista de dataframes
dataframes = sorted(dataframes, key=lambda x: x['Station'].iloc[0])

# Se crea el date range con las fechas inicial y final conocidas, para presión atmosférica 01/01/2001 a 03/04/2024, horaria
date_range = pd.date_range(start="2005-01-01 00:00", end="2024-10-15 23:00", freq='h')

# Número de estaciones por gráfico
estaciones_por_grafico = 50

# Total de gráficos a generar
total_graficos = len(dataframes) // estaciones_por_grafico + (len(dataframes) % estaciones_por_grafico > 0)

# Configuración de la fuente para todo el gráfico
font = {'family': 'Franklin Gothic Book',
        'weight': 'normal',
        'size': 10}

plt.rc('font', **font)

for j in range(total_graficos):
    fig, ax = plt.subplots(figsize=(15, 10))
    inicio = j * estaciones_por_grafico
    fin = min(inicio + estaciones_por_grafico, len(dataframes))  # Asegurarse de no pasarse del rango

    # Define a helper function to plot data for clarity
    def plot_station_data(ax, data, index, station_label):
        # Find where data is present
        presence_mask = data['Presence'] == 1
        # Plot only where data is present
        ax.fill_between(data.index, index - 0.45, index + 0.45, where=~presence_mask, color='#f0f0f0', step='mid', label='Sin dato' if i == 1 else "")
        ax.fill_between(data.index, index - 0.45, index + 0.45, where=presence_mask, color='#612b06', step='mid', label='Con dato' if i == 1 else "")

    for i, df in enumerate(dataframes[inicio:fin], start=1):
        # Check for duplicates
        duplicates = df.index.duplicated()
        # Optionally, drop or keep the first/last occurrence of duplicates
        if duplicates.any():
            df = df.loc[~duplicates]
        # Reindex the dataframe to the full date range, filling missing data with 0 (indicating absence)
        df_complete = df.reindex(date_range, fill_value=0)
        plot_station_data(ax, df_complete, i, f"Station {df['Station'].iloc[0]}")

    #ax.legend(loc='upper left')  # Agrega la leyenda en la esquina superior izquierda
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=2)
    # Setting the labels and ticks for better readability
    ax.set_yticks(range(1, fin - inicio + 1))
    ax.set_yticklabels([f"{df['Station'].iloc[0]}" for df in dataframes[inicio:fin]], fontsize=8)
    ax.set_ylim(0.5, fin - inicio + 0.5)
    ax.set_xlim(date_range[0], date_range[-1])
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=90)
    plt.ylabel('Código de estación CNE')
    plt.title(f'Completitud de datos por estación - Grupo de estaciones {j+1} de {total_graficos}')
    plt.grid(True, which='both', linestyle='-', linewidth=0.25)

    # Show and save the plot
    plt.tight_layout()
    plt.savefig(f'completitud_datos_grafico_{j+1}.png')  # Guarda el gráfico como un archivo PNG
    plt.close(fig)  # Cierra la figura para liberar memoria

In [18]:
from matplotlib import font_manager
fonts = [f.name for f in font_manager.fontManager.ttflist]
print(set(fonts))

{'Franklin Gothic Demi', 'Cambria', 'Lucida Sans Unicode', 'Stencil', 'Niagara Engraved', 'Harlow Solid Italic', 'Calibri', 'ESRI AMFM Water', 'Haettenschweiler', 'Microsoft PhagsPa', 'ESRI Meteorological 01', 'Curlz MT', 'Oswald', 'Century', 'DejaVu Sans Display', 'cmex10', 'Microsoft Tai Le', 'Chiller', 'Gigi', 'Maiandra GD', 'ESRI Pipeline US 1', 'Bradley Hand ITC', 'Edwardian Script ITC', 'SimSun-ExtB', 'HoloLens MDL2 Assets', 'ESRI IGL Font16', 'High Tower Text', 'ESRI SDS 1.95 1', 'Pristina', 'Myanmar Text', 'MingLiU-ExtB', 'ESRI Telecom', 'Lucida Handwriting', 'Harrington', 'Rockwell Extra Bold', 'Palace Script MT', 'Vivaldi', 'ESRI MilSym 03', 'Trebuchet MS', 'ESRI AMFM Electric', 'MT Extra', 'ESRI Weather', 'Playbill', 'ESRI IGL Font23', 'ESRI US MUTCD 3', 'Imprint MT Shadow', 'Segoe UI Emoji', 'DejaVu Serif Display', 'Perpetua', 'ESRI Commodities', 'Rockwell', 'ESRI Surveyor', 'Franklin Gothic Heavy', 'Dosis', 'Franklin Gothic Demi Cond', 'STIXSizeFiveSym', 'Tw Cen MT Condens

### Cantidad total de datos

In [20]:
## Se indaga sobre cuál es la cantidad de datos de todas las series descargadas en el período 2001-01-01 al 2024-03-31
# Se crea la función cantidad_datos
def cantidad_datos(archivos):
    total_filas = 0

    for archivo in archivos:
        # Se utiliza el método 'chunksize' de pandas para leer los archivos por partes (chunks)
        # y procesarlos de forma incremental sin cargar todos los datos en memoria al mismo tiempo
        chunks = pd.read_csv(archivo, encoding='latin-1', chunksize=100000)  # Se establece este valor de chunk para poder hacer otras actividades

        for chunk in chunks:
            total_filas += len(chunk)

    return total_filas

# Ruta de la carpeta con los archivos CSV
carpeta_proces = "../OE_3_QC_Variables/3_TemperaturaSuelo/TS50/RawUnmodified_TS50/"

# Obtener la lista de archivos en la carpeta
archivos = [carpeta_proces + archivo for archivo in os.listdir(carpeta_proces) if archivo.endswith('.csv')]

# Llamar a la función para contar las filas
total_filas = cantidad_datos(archivos)

# Imprimir el resultado
print("El número total de datos es:", total_filas)

El número total de datos es: 6457985


### Contar datos por fecha

In [22]:
def contar_datos_por_fechas(archivos):
    resultados = []

    for archivo in archivos:
        chunks = pd.read_csv(archivo, encoding='latin-1', chunksize=100000)
        
        for chunk in chunks:
            chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
            estacion = chunk['Station'].iloc[0]
            
            # Conteo por año, mes y día
            conteo_dia = chunk['Fecha'].dt.to_period('D').value_counts().rename_axis('Año-mes-dia').reset_index(name='Cantidad_año-mes-dia')
            conteo_dia['Año-mes'] = conteo_dia['Año-mes-dia'].dt.to_timestamp().dt.to_period('M')
            conteo_dia['Año'] = conteo_dia['Año-mes'].dt.year

            # Conteo por año y mes
            conteo_mes = chunk['Fecha'].dt.to_period('M').value_counts().rename_axis('Año-mes').reset_index(name='Cantidad_año-mes')
            conteo_mes['Año'] = conteo_mes['Año-mes'].dt.year

            # Conteo por año
            conteo_anual = chunk['Fecha'].dt.year.value_counts().rename_axis('Año').reset_index(name='Cantidad_año')
            
            # Combinar con la información de la estación
            conteo_anual['Station'] = estacion
            conteo_mes['Station'] = estacion
            conteo_dia['Station'] = estacion
            
            # Merge considerando Station, Año y Año-mes
            merged_data = pd.merge(pd.merge(conteo_anual, conteo_mes, on=['Station', 'Año']), conteo_dia, on=['Station', 'Año', 'Año-mes'])
            resultados.append(merged_data)
    
    df_resultados = pd.concat(resultados, ignore_index=True)
    # Reordenar columnas
    df_resultados = df_resultados[['Station', 'Año', 'Cantidad_año', 'Año-mes', 'Cantidad_año-mes','Año-mes-dia','Cantidad_año-mes-dia']]
    df_resultados.sort_values(by=['Station', 'Año', 'Año-mes', 'Año-mes-dia'], inplace=True)
    
    # Rellenar la columna 'Cantidad_año' sólo una vez por cada año y estación
    df_resultados['Cantidad_año'] = df_resultados.groupby(['Station', 'Año'])['Cantidad_año'].transform(lambda x: x.iloc[0] if not x.isna().all() else pd.NA)
    
    df_resultados.to_csv('ConteoDatos_AnioMesDia_TS50.csv', index=False, encoding='latin-1')
    return df_resultados

# Uso de la función
carpeta_proces = "../OE_3_QC_Variables/3_TemperaturaSuelo/TS50/RawUnmodified_TS50/"
archivos = [carpeta_proces + archivo for archivo in os.listdir(carpeta_proces) if archivo.endswith('.csv')]
resultados = contar_datos_por_fechas(archivos)
print(resultados.head())

       Station   Año  Cantidad_año  Año-mes  Cantidad_año-mes Año-mes-dia  \
1637  11105020  2005           775  2005-08                 4  2005-08-31   
1636  11105020  2005           775  2005-09                 9  2005-09-26   
1624  11105020  2005           775  2005-10               593  2005-10-05   
1626  11105020  2005           775  2005-10               593  2005-10-06   
1625  11105020  2005           775  2005-10               593  2005-10-07   

      Cantidad_año-mes-dia  
1637                     4  
1636                     9  
1624                     9  
1626                     5  
1625                     6  


In [7]:
### Lectura de frecuencias para revisión inicial
dffreq = pd.read_csv('../../OE_2_DiagnosticoActualDatos/An8_ConteoDatos_AnioMesDia_HR.csv', encoding='latin-1')
dffreq

,Station,Año,Cantidad_año,Año-mes,Cantidad_año-mes,Año-mes-dia,Cantidad_año-mes-dia
0,11025501,2016,32894,2016-06,7589,2016-06-02,113
1,11025501,2016,32894,2016-06,7589,2016-06-03,204
2,11025501,2016,32894,2016-06,7589,2016-06-05,84
3,11025501,2016,32894,2016-06,7589,2016-06-06,288
4,11025501,2016,32894,2016-06,7589,2016-06-07,288
...,...,...,...,...,...,...,...
1293409,5311500149,2024,8244,2024-03,622,2024-03-01,138
1293410,5311500149,2024,8244,2024-03,622,2024-03-02,144
1293411,5311500149,2024,8244,2024-03,622,2024-03-03,144
1293412,5311500149,2024,8244,2024-03,622,2024-03-04,144


In [8]:
grupito = dffreq.groupby(dffreq['Station'])

In [10]:
grp_mx = grupito['Cantidad_año'].max()
grp_mx.to_csv('grp_mx.csv', encoding='latin-1', index=True, sep=';')

### Obtener fechas iniciales y finales de estaciones

In [24]:
def obtener_fechas(archivo):
    try:
        # Se lee el archivo
        datos = pd.read_csv(archivo, encoding='latin-1')#, names=['Fecha', 'Valor'])
        try:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S')
        
        # Se obtiene el código de la estación
        station = datos['Station'].values[0]
        
        # Se obtiene primera y última fecha 
        fecha_inicial = datos['Fecha'].iloc[0]
        fecha_final = datos['Fecha'].iloc[-1]
        
        return station, fecha_inicial, fecha_final
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        return None, None, None

def main():
    # Cambia la ruta según la ubicación de tu carpeta
    ruta_carpeta = "../OE_3_QC_Variables/3_TemperaturaSuelo/TS50/RawUnmodified_TS50"

    # Listamos todos los archivos en la carpeta que terminen con .csv
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    # Creamos una lista para almacenar los resultados
    resultados = []

    # Iteramos sobre cada archivo y obtenemos las fechas
    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, fecha_inicial, fecha_final = obtener_fechas(ruta_archivo)
        
        if station and fecha_inicial and fecha_final:
            resultados.append([station, fecha_inicial, fecha_final])
    
    # Convertimos los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados, columns=['CodEstacion', 'fecha_inicial', 'fecha_final'])
    
    # Guardamos el DataFrame como un archivo CSV
    resultados_df.to_csv('FechaInicialFinal_TS50.csv', index=False)

if __name__ == "__main__":
    main()

### Obtener estadísticos descriptivos

In [25]:
def obtener_EstadDescript(archivo):
    try:
        # Se lee el archivo
        datos = pd.read_csv(archivo, encoding='latin-1')#, names=['Fecha', 'Valor'])
        try:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S')
        
        # Se obtiene el código de la estación
        station = datos['Station'].values[0]
        
        # Se obtienen los estadísticos descriptivos
        minimo = datos['Valor'].min()
        maximo = datos['Valor'].max()
        media = datos['Valor'].mean()
        mediana = datos['Valor'].median()
        desvest = datos['Valor'].std()
        varianza = datos['Valor'].var()
        #first_q = datos['Valor'].quantile(0.25)
        #third_q = datos['Valor'].quantile(0.75)
        
        return station, minimo, maximo, media, mediana, desvest, varianza, #first_q, third_q, mediana --no fue posible calcular estos estad.
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        print(e.__class__)
        return None, None, None, None, None, None, None#, None, None --no fue posible calcular estos estad.

def main():
    # Cambia la ruta según la ubicación de tu carpeta
    ruta_carpeta = "../OE_3_QC_Variables/3_TemperaturaSuelo/TS50/RawUnmodified_TS50"

    # Listamos todos los archivos en la carpeta que terminen con .csv
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    # Se crea una lista para almacenar los resultados
    resultados = []

    # Se itera sobre cada archivo y se obtienen fechas las fechas
    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, minimo, maximo, media, mediana, desvest, varianza= obtener_EstadDescript(ruta_archivo)
        print(station, minimo, maximo, media, mediana, desvest, varianza)
        resultados.append([station, minimo, maximo, media, mediana, desvest, varianza])
        #if station and minimo: #and maximo and media and mediana and desvest and varianza:
            #resultados.append([station, minimo, maximo, media, mediana, desvest, varianza])
        #else:
            #print(f'Sin suficientes resultado para {archivo}. No se generaron sus estadísticos')
    
    # Se convierten los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados, columns=['Station', 'minimo', 'maximo', 'media', 'mediana','desvest', 'varianza'])
    
    print(resultados_df)
    
    # Se guarda el DataFrame como un archivo CSV
    resultados_df.to_csv('EstadDescript_TS50_Raw.csv', index=False)

if __name__ == "__main__":
    main()

11105020 -246.4 3277.8 21.58186649326188 27.5 28.128050684323217 791.1872352998557
11135030 0.0 568.1 28.888087887796328 30.0 15.584572100043246 242.87888754144637
12015100 -445.3 3277.8 28.368772390733223 28.2 13.153216061909914 173.00709277128496
12015110 -71.3 1177.6 29.765625672283967 29.4 5.9068490390931 34.89086557063507
13085050 -12530.7 11013.8 30.799208695045035 30.2 118.81114804160894 14116.088898965116
15085050 -480.8 3277.9 34.79081428484812 33.0 38.017740091927735 1445.3485616973696
16055120 -3046.4 3277.9 47.52339402267261 29.1 59.580722467332066 3549.8624897292484
21015050 11.5 3277.8 19.84185317670453 19.8 18.00624879857761 324.2249957962776
21015070 -1804.2 4511.6 19.094488111046047 18.9 31.16106552900361 971.0120049028571
21115010 12.8 3277.8 31.2451450855972 31.1 26.458082475771246 700.0301282947134
21115180 -3123.2 3277.8 30.07879059595408 29.5 24.38060810056763 594.414051353464
21195170 -3234.8 3277.8 127.6340367688198 14.0 368.1983331469181 135570.0125321689
21195

In [15]:
resultados_df = pd.read_csv('EstadDescript_TS10_Raw.csv')

In [12]:
altipatm = pd.read_table('EMA_Patm_TMaxMin.txt', sep=';')

In [13]:
altipatm.head(2)

,OBJECTID,Station,lat,long_,nombre,sede,id_rule,codigo,Instituc,FreqInf,Altitud,TmaxNorm,TminNorm
0,1,11030010.0,5.375,-76.613,Valle,09 - Cali,9,9,IDEAM,H,58,27.853392,19.794912
1,2,11035030.0,5.285,-76.628,Valle,09 - Cali,9,9,IDEAM,H,104,27.376471,19.418962


In [14]:
estadalt = pd.merge(resultados_df, altipatm[['Station','Altitud']], on='Station')

In [16]:
# Se sobreescriben los estadísticos para incluir altitud
estadalt.to_csv('EstadDescript_Patm_Raw.csv', index=False)

In [ ]:
# De datos con QC
def obtener_EstadDescript(archivo):
    try:
        # Se lee el archivo
        datos = pd.read_csv(archivo, encoding='latin-1')#, names=['Fecha', 'Valor'])
        try:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
            datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%Y-%m-%d %H:%M:%S')
        
        # Se hace el filtro para que solo queden los valores que superaron las pruebas
        dfC = datos[datos['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))]
        
        # Se obtiene el código de la estación
        station = dfC['Station'].values[0]
        
        # Se obtienen los estadísticos descriptivos
        minimo = dfC['Valor'].min()
        maximo = dfC['Valor'].max()
        media = dfC['Valor'].mean()
        #mediana = datos['Valor'].median()
        desvest = dfC['Valor'].std()
        varianza = dfC['Valor'].var()
        #first_q = datos['Valor'].quantile(0.25)
        #third_q = datos['Valor'].quantile(0.75)
        
        return station, minimo, maximo, media, desvest, varianza, #first_q, third_q, mediana --no fue posible calcular estos estad.
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        print(e.__class__)
        return None, None, None, None, None, None#, None, None, None --no fue posible calcular estos estad.

def main():
    # Cambia la ruta según la ubicación de tu carpeta
    ruta_carpeta = 'DatosEjemplo'

    # Listamos todos los archivos en la carpeta que terminen con .csv
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    # Creamos una lista para almacenar los resultados
    resultados = []

    # Iteramos sobre cada archivo y obtenemos las fechas
    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, minimo, maximo, media, desvest, varianza= obtener_EstadDescript(ruta_archivo)
        
        if station and minimo and maximo and media and desvest and varianza:
            resultados.append([station, minimo, maximo, media, desvest, varianza])
    
    # Convertimos los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados, columns=['Station', 'minimo', 'maximo', 'media', 'desvest', 'varianza'])
    
    # Guardamos el DataFrame como un archivo CSV
    resultados_df.to_csv('EstadDescript_Patm_QC.csv', index=False)

if __name__ == "__main__":
    main()

_____